In [1]:
import cv2
import numpy as np
import os
from ultralytics import YOLO

# --- 1. AYARLAR VE MODELLER ---
PANO_MODEL_PATH = r"C:\Users\User\runs\classify\pano_denetim_v12\weights\best.pt"
PERSON_MODEL_PATH = 'yolov8n.pt'
VIDEO_PATH = "test_yetkili.mp4"

# Koordinatlar
PANO_PTS = np.float32([[200, 620], [420, 660], [430, 780], [220, 750]])
YASAK_YOL_PTS = np.array([[1400, 10], [1250, 10], [1080, 1080], [1730, 1080]], np.int32)

# Eşik Değerleri
GREEN_RATIO_THRESHOLD = 0.05
PERSON_CONF = 0.50

# Klasörleme
desktop = os.path.join(os.path.expanduser("~"), "Desktop")
ihlal_path = os.path.join(desktop, "ihlal")
if not os.path.exists(ihlal_path): os.makedirs(ihlal_path)

# Modelleri Yükle
pano_model = YOLO(PANO_MODEL_PATH)
person_model = YOLO(PERSON_MODEL_PATH)

# Hafıza Yönetimi
authorized_ids = set() 
person_history = {}    
last_boxes = {}        
ihlal_kaydedilen_idlar = set() # Yasak yol için
pano_ihlal_kaydedildi = False  # Pano ihlali için (kare bazlı değil durum bazlı)
next_id = 1

cap = cv2.VideoCapture(VIDEO_PATH)
w, h = int(cap.get(3)), int(cap.get(4))
out = cv2.VideoWriter('genel_output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 20.0, (w, h))

def check_green_locally(person_img):
    if person_img is None or person_img.size == 0: return False
    ih, iw = person_img.shape[:2]
    upper = person_img[int(ih*0.1):int(ih*0.45), :]
    hsv = cv2.cvtColor(upper, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, np.array([30, 40, 40]), np.array([95, 255, 255]))
    return (np.sum(mask > 0) / mask.size) > GREEN_RATIO_THRESHOLD

frame_idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break

    # --- A. PANO DURUM KONTROLÜ ---
    dst_pts = np.float32([[0, 0], [224, 0], [224, 224], [0, 224]])
    matrix = cv2.getPerspectiveTransform(PANO_PTS, dst_pts)
    pano_crop = cv2.warpPerspective(frame, matrix, (224, 224))
    pano_results = pano_model.predict(pano_crop, verbose=False)[0]
    pano_label = pano_results.names[pano_results.probs.top1]
    pano_conf = pano_results.probs.top1conf.item()

    # --- B. PERSONEL TAKİBİ VE YETKİ ---
    results = person_model.predict(frame, conf=PERSON_CONF, imgsz=640, classes=[0], verbose=False)
    current_frame_ids = []
    authorized_present = False # Panonun başında yetkili var mı?

    if len(results[0].boxes) > 0:
        for box in results[0].boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
            
            # ID Takibi
            assigned_id = None
            for pid, last_pos in person_history.items():
                if np.sqrt((cx - last_pos[0])**2 + (cy - last_pos[1])**2) < 80:
                    assigned_id = pid
                    break
            if assigned_id is None:
                assigned_id = next_id
                next_id += 1
            
            person_history[assigned_id] = (cx, cy)
            last_boxes[assigned_id] = (x1, y1, x2, y2)
            current_frame_ids.append(assigned_id)

            # Yetki Kontrolü
            if assigned_id not in authorized_ids:
                crop = frame[max(0, y1):min(h, y2), max(0, x1):min(w, x2)]
                if check_green_locally(crop): authorized_ids.add(assigned_id)
            
            if assigned_id in authorized_ids: authorized_present = True

            # --- C. YASAK YOL İHLALİ ---
            if cv2.pointPolygonTest(YASAK_YOL_PTS, (float(cx), float(y2)), False) >= 0:
                if assigned_id not in ihlal_kaydedilen_idlar:
                    cv2.imwrite(os.path.join(ihlal_path, f"yasak_yol_id_{assigned_id}.jpg"), frame)
                    ihlal_kaydedilen_idlar.add(assigned_id)

    # --- D. MANTIK VE GÖRSELLEŞTİRME ---
    # 1. Pano Çizimi
    pano_color = (0, 255, 0) if pano_label == "kapali" else (0, 0, 255)
    cv2.polylines(frame, [PANO_PTS.astype(np.int32)], True, pano_color, 3)
    cv2.putText(frame, f"Pano: {pano_label.upper()}", (int(PANO_PTS[0][0]), int(PANO_PTS[0][1]-10)), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, pano_color, 2)

    # 2. Yasak Yol Çizimi
    overlay = frame.copy()
    cv2.fillPoly(overlay, [YASAK_YOL_PTS], (0, 0, 255))
    cv2.addWeighted(overlay, 0.15, frame, 0.85, 0, frame)

    # 3. Pano İhlal Senaryosu
    if pano_label == "acik" and not authorized_present:
        cv2.putText(frame, "!!! YETKISIZ PANO MUDHALESI !!!", (50, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)
        if not pano_ihlal_kaydedildi:
            cv2.imwrite(os.path.join(ihlal_path, f"pano_ihlal_frame_{frame_idx}.jpg"), frame)
            pano_ihlal_kaydedildi = True
    elif pano_label == "kapali":
        pano_ihlal_kaydedildi = False # Pano kapanınca ihlal sayacı sıfırlanır

    # 4. Personel Kutuları
    for pid in current_frame_ids:
        x1, y1, x2, y2 = last_boxes[pid]
        is_auth = pid in authorized_ids
        box_color = (0, 255, 0) if is_auth else (0, 0, 255)
        label = f"ID:{pid} {'YETKILI' if is_auth else 'PERSONEL'}"
        cv2.rectangle(frame, (x1, y1), (x2, y2), box_color, 2)
        cv2.putText(frame, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, box_color, 2)

    out.write(frame)
    frame_idx += 1

cap.release()
out.release()
print(f"✅ Sistem başarıyla çalıştı. İhlaller {ihlal_path} klasöründe.")

✅ Sistem başarıyla çalıştı. İhlaller C:\Users\User\Desktop\ihlal klasöründe.


In [4]:
import cv2
import numpy as np
import os
from ultralytics import YOLO

# --- 1. AYARLAR VE MODELLER ---
PANO_MODEL_PATH = r"C:\Users\User\runs\classify\pano_denetim_v12\weights\best.pt"
PERSON_MODEL_PATH = 'yolov8n.pt'
VIDEO_PATH = "test_yetkili.mp4"

# Koordinatlar
PANO_PTS = np.float32([[200, 620], [420, 660], [430, 780], [220, 750]])
YASAK_YOL_PTS = np.array([[1400, 10], [1250, 10], [1080, 1080], [1730, 1080]], np.int32)

# Mesafe ve Eşik Değerleri
DISTANCE_OFFSET = 300       # Pano etrafındaki hayali etki alanı (Piksel)
GREEN_RATIO_THRESHOLD = 0.05 # Yeşil yelek hassasiyeti (Biraz düşürüldü)
PERSON_CONF = 0.50         # Daha hassas insan tespiti

# Klasörleme
desktop = os.path.join(os.path.expanduser("~"), "Desktop")
ihlal_path = os.path.join(desktop, "ihlal")
if not os.path.exists(ihlal_path): os.makedirs(ihlal_path)

# Modelleri Yükle
pano_model = YOLO(PANO_MODEL_PATH)
person_model = YOLO(PERSON_MODEL_PATH)

# Hafıza Yönetimi
authorized_ids = set() 
person_history = {}    
last_boxes = {}        
ihlal_kaydedilen_idlar = set()
pano_ihlal_kaydedildi = False
next_id = 1

cap = cv2.VideoCapture(VIDEO_PATH)
w, h = int(cap.get(3)), int(cap.get(4))
out = cv2.VideoWriter('genel_output_v2.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 20.0, (w, h))

def check_green_locally(person_img):
    if person_img is None or person_img.size == 0: return False
    ih, iw = person_img.shape[:2]
    upper = person_img[int(ih*0.1):int(ih*0.45), :]
    hsv = cv2.cvtColor(upper, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, np.array([30, 40, 40]), np.array([95, 255, 255]))
    return (np.sum(mask > 0) / mask.size) > GREEN_RATIO_THRESHOLD

frame_idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break

    # --- A. PANO DURUM KONTROLÜ ---
    dst_pts = np.float32([[0, 0], [224, 0], [224, 224], [0, 224]])
    matrix = cv2.getPerspectiveTransform(PANO_PTS, dst_pts)
    pano_crop = cv2.warpPerspective(frame, matrix, (224, 224))
    pano_results = pano_model.predict(pano_crop, verbose=False)[0]
    pano_label = pano_results.names[pano_results.probs.top1]

    # --- B. PERSONEL TAKİBİ VE YETKİ ---
    results = person_model.predict(frame, conf=PERSON_CONF, imgsz=640, classes=[0], verbose=False)
    current_frame_ids = []
    
    # Pano Etrafındaki Yetki Kontrolü İçin Değişkenler
    p_min_x, p_max_x = np.min(PANO_PTS[:, 0]) - DISTANCE_OFFSET, np.max(PANO_PTS[:, 0]) + DISTANCE_OFFSET
    p_min_y, p_max_y = np.min(PANO_PTS[:, 1]) - DISTANCE_OFFSET, np.max(PANO_PTS[:, 1]) + DISTANCE_OFFSET
    
    authorized_near_pano = False
    person_near_pano = False

    if len(results[0].boxes) > 0:
        for box in results[0].boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
            
            # Manuel ID Takibi
            assigned_id = None
            for pid, last_pos in person_history.items():
                if np.sqrt((cx - last_pos[0])**2 + (cy - last_pos[1])**2) < 100:
                    assigned_id = pid
                    break
            if assigned_id is None:
                assigned_id = next_id
                next_id += 1
            
            person_history[assigned_id] = (cx, cy)
            last_boxes[assigned_id] = (x1, y1, x2, y2)
            current_frame_ids.append(assigned_id)

            # Yeşil Yelek/Yetki Kontrolü
            if assigned_id not in authorized_ids:
                crop = frame[max(0, y1):min(h, y2), max(0, x1):min(w, x2)]
                if check_green_locally(crop): authorized_ids.add(assigned_id)
            
            # Panoya Yakınlık Kontrolü
            if (p_min_x < cx < p_max_x) and (p_min_y < cy < p_max_y):
                person_near_pano = True
                if assigned_id in authorized_ids:
                    authorized_near_pano = True

            # Yasak Yol İhlali
            if cv2.pointPolygonTest(YASAK_YOL_PTS, (float(cx), float(y2)), False) >= 0:
                if assigned_id not in ihlal_kaydedilen_idlar:
                    cv2.imwrite(os.path.join(ihlal_path, f"yasak_yol_id_{assigned_id}.jpg"), frame)
                    ihlal_kaydedilen_idlar.add(assigned_id)

    # --- C. MANTIK VE GÖRSELLEŞTİRME ---
    
    # 1. Pano Bölgesi ve Etki Alanı Çizimi (Genişletilmiş Alan)
    pano_color = (0, 255, 0) if pano_label == "kapali" else (0, 0, 255)
    cv2.polylines(frame, [PANO_PTS.astype(np.int32)], True, pano_color, 3)
    # Debug: Panonun etki alanını görmek için kesik çizgi (Opsiyonel)
    cv2.rectangle(frame, (int(p_min_x), int(p_min_y)), (int(p_max_x), int(p_max_y)), (255, 255, 0), 1)

    # 2. Yasak Yol ve Pano İhlal Metinleri
    if pano_label == "acik" and person_near_pano and not authorized_near_pano:
        cv2.putText(frame, "!!! YETKISIZ MUDAHALE !!!", (50, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)
        if not pano_ihlal_kaydedildi:
            cv2.imwrite(os.path.join(ihlal_path, f"pano_ihlal_kare_{frame_idx}.jpg"), frame)
            pano_ihlal_kaydedildi = True
    elif pano_label == "kapali" or authorized_near_pano:
        pano_ihlal_kaydedildi = False

    # 3. Yasak Yol Görselleştirme
    overlay = frame.copy()
    cv2.fillPoly(overlay, [YASAK_YOL_PTS], (0, 0, 255))
    cv2.addWeighted(overlay, 0.15, frame, 0.85, 0, frame)

    # 4. Aktif Personelleri Çiz
    for pid in current_frame_ids:
        x1, y1, x2, y2 = last_boxes[pid]
        is_auth = pid in authorized_ids
        box_color = (0, 255, 0) if is_auth else (0, 0, 255)
        label = f"ID:{pid} {'YETKILI' if is_auth else 'PERSONEL'}"
        cv2.rectangle(frame, (x1, y1), (x2, y2), box_color, 2)
        cv2.putText(frame, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, box_color, 2)

    out.write(frame)
    frame_idx += 1

cap.release()
out.release()
print(f"✅ Analiz tamamlandı. Mesafe toleransı: {DISTANCE_OFFSET} piksel.")

✅ Analiz tamamlandı. Mesafe toleransı: 300 piksel.


In [10]:
import cv2
import numpy as np
import os
from ultralytics import YOLO

# --- 1. AYARLAR VE MODELLER ---
PANO_MODEL_PATH = r"C:\Users\User\runs\classify\pano_denetim_v12\weights\best.pt"
PERSON_MODEL_PATH = 'yolov8n.pt'
VIDEO_PATH = "test_yetkili.mp4"

# Koordinatlar (Senin belirttiğin sabit değerler)
PANO_PTS = np.float32([[200, 620], [420, 660], [430, 780], [220, 750]])
YASAK_YOL_PTS = np.array([[1400, 10], [1250, 10], [1080, 1080], [1730, 1080]], np.int32)

# Mesafe ve Eşik Değerleri
DISTANCE_OFFSET = 450       # Pano etrafındaki hayali etki alanı (Piksel bazlı genişletme)
GREEN_RATIO_THRESHOLD = 0.10 # Yeşil yelek hassasiyeti
PERSON_CONF = 0.50          # İnsan tespiti güven eşiği

# Klasörleme
desktop = os.path.join(os.path.expanduser("~"), "Desktop")
ihlal_path = os.path.join(desktop, "ihlal")
if not os.path.exists(ihlal_path): os.makedirs(ihlal_path)

# Modelleri Yükle
pano_model = YOLO(PANO_MODEL_PATH)
person_model = YOLO(PERSON_MODEL_PATH)

# Hafıza Yönetimi
authorized_ids = set() 
person_history = {}    
last_boxes = {}        
ihlal_kaydedilen_idlar = set()
pano_ihlal_kaydedildi = False
next_id = 1

cap = cv2.VideoCapture(VIDEO_PATH)
w, h = int(cap.get(3)), int(cap.get(4))
out = cv2.VideoWriter('genel_3.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 20.0, (w, h))

def check_green_locally(person_img):
    if person_img is None or person_img.size == 0: return False
    ih, iw = person_img.shape[:2]
    # Üst gövdeye odaklan (yelek bölgesi)
    upper = person_img[int(ih*0.1):int(ih*0.45), :]
    hsv = cv2.cvtColor(upper, cv2.COLOR_BGR2HSV)
    # Fosforlu yeşil/sarı maskesi
    mask = cv2.inRange(hsv, np.array([25, 40, 40]), np.array([95, 255, 255]))
    return (np.sum(mask > 0) / mask.size) > GREEN_RATIO_THRESHOLD

frame_idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break

    # --- A. PANO DURUM KONTROLÜ ---
    dst_pts = np.float32([[0, 0], [224, 0], [224, 224], [0, 224]])
    matrix = cv2.getPerspectiveTransform(PANO_PTS, dst_pts)
    pano_crop = cv2.warpPerspective(frame, matrix, (224, 224))
    pano_results = pano_model.predict(pano_crop, verbose=False)[0]
    pano_label = pano_results.names[pano_results.probs.top1]

    # --- B. PERSONEL TAKİBİ VE BÖLGE ANALİZİ ---
    results = person_model.predict(frame, conf=PERSON_CONF, imgsz=640, classes=[0], verbose=False)
    current_frame_ids = []
    
    # Pano Etki Alanı Sınırları (Esnek Mesafe)
    p_min_x, p_max_x = np.min(PANO_PTS[:, 0]) - DISTANCE_OFFSET, np.max(PANO_PTS[:, 0]) + DISTANCE_OFFSET
    p_min_y, p_max_y = np.min(PANO_PTS[:, 1]) - DISTANCE_OFFSET, np.max(PANO_PTS[:, 1]) + DISTANCE_OFFSET
    
    any_person_near_pano = False
    any_authorized_near_pano = False

    if len(results[0].boxes) > 0:
        for box in results[0].boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
            
            # --- MANUEL ID TAKİBİ ---
            assigned_id = None
            for pid, last_pos in person_history.items():
                if np.sqrt((cx - last_pos[0])**2 + (cy - last_pos[1])**2) < 100:
                    assigned_id = pid
                    break
            if assigned_id is None:
                assigned_id = next_id
                next_id += 1
            
            person_history[assigned_id] = (cx, cy)
            last_boxes[assigned_id] = (x1, y1, x2, y2)
            current_frame_ids.append(assigned_id)

            # --- YETKİ VE İHLAL MANTIĞI ---
            if assigned_id not in authorized_ids:
                crop = frame[max(0, y1):min(h, y2), max(0, x1):min(w, x2)]
                if check_green_locally(crop): authorized_ids.add(assigned_id)
            
            # Pano Bölgesiyle Kesişim Kontrolü (Box bazlı)
            if not (x2 < p_min_x or x1 > p_max_x or y2 < p_min_y or y1 > p_max_y):
                any_person_near_pano = True
                if assigned_id in authorized_ids:
                    any_authorized_near_pano = True

            # Yasak Yol Kontrolü
            if cv2.pointPolygonTest(YASAK_YOL_PTS, (float(cx), float(y2)), False) >= 0:
                if assigned_id not in ihlal_kaydedilen_idlar:
                    cv2.imwrite(os.path.join(ihlal_path, f"yasak_yol_ID_{assigned_id}.jpg"), frame)
                    ihlal_kaydedilen_idlar.add(assigned_id)

    # --- C. GÖRSELLEŞTİRME VE KARAR ---
    
    # 1. Pano Durumu
    pano_color = (0, 255, 0) if pano_label == "kapali" else (0, 0, 255)
    cv2.polylines(frame, [PANO_PTS.astype(np.int32)], True, pano_color, 3)
    cv2.putText(frame, f"PANO: {pano_label.upper()}", (int(PANO_PTS[0][0]), int(PANO_PTS[0][1]-15)), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, pano_color, 2)

    # 2. Pano İhlal Kararı
    if pano_label == "acik" and any_person_near_pano and not any_authorized_near_pano:
        cv2.putText(frame, "!!! YETKISIZ PANO MUDAHALESI !!!", (50, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 4)
        if not pano_ihlal_kaydedildi:
            cv2.imwrite(os.path.join(ihlal_path, f"pano_ihlal_F_{frame_idx}.jpg"), frame)
            pano_ihlal_kaydedildi = True
    elif pano_label == "kapali" or any_authorized_near_pano:
        pano_ihlal_kaydedildi = False

    # 3. Bölgeleri Çiz
    overlay = frame.copy()
    cv2.fillPoly(overlay, [YASAK_YOL_PTS], (0, 0, 255)) # Yasak Yol
    cv2.rectangle(frame, (int(p_min_x), int(p_min_y)), (int(p_max_x), int(p_max_y)), (255, 255, 0), 1) # Pano Alanı
    cv2.addWeighted(overlay, 0.15, frame, 0.85, 0, frame)

    # 4. Personel Kutuları
    for pid in current_frame_ids:
        x1, y1, x2, y2 = last_boxes[pid]
        is_auth = pid in authorized_ids
        color = (0, 255, 0) if is_auth else (0, 0, 255)
        label = f"ID:{pid} {'YETKILI' if is_auth else 'PERSONEL'}"
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)
        cv2.putText(frame, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    out.write(frame)
    frame_idx += 1

cap.release()
out.release()
print(f"✅ İşlem tamamlandı. Mesafe toleransı {DISTANCE_OFFSET} piksel olarak güncellendi.")

✅ İşlem tamamlandı. Mesafe toleransı 450 piksel olarak güncellendi.


In [12]:
import cv2
import numpy as np
import os
from ultralytics import YOLO

# --- 1. AYARLAR ---
PANO_MODEL_PATH = r"C:\Users\User\runs\classify\pano_denetim_v12\weights\best.pt"
PERSON_MODEL_PATH = 'yolov8n.pt'
VIDEO_PATH = "test_yetkili.mp4"

PANO_PTS = np.float32([[200, 620], [420, 660], [430, 780], [220, 750]])
YASAK_YOL_PTS = np.array([[1400, 10], [1250, 10], [1080, 1080], [1730, 1080]], np.int32)

DISTANCE_OFFSET = 500       # Toleransı biraz daha artırdım (500px)
GREEN_RATIO_THRESHOLD = 0.12 # Hassasiyeti artırdım (0.10 çok katıydı)
PERSON_CONF = 0.50         # Kişi yakalamayı kolaylaştırdım

# Klasörleme
desktop = os.path.join(os.path.expanduser("~"), "Desktop")
ihlal_path = os.path.join(desktop, "ihlal")
if not os.path.exists(ihlal_path): os.makedirs(ihlal_path)

pano_model = YOLO(PANO_MODEL_PATH)
person_model = YOLO(PERSON_MODEL_PATH)

# Hafıza
authorized_ids = set() 
person_history = {}    
last_boxes = {}        
ihlal_kaydedilen_idlar = set()
pano_ihlal_kaydedildi = False
next_id = 1

cap = cv2.VideoCapture(VIDEO_PATH)
w, h = int(cap.get(3)), int(cap.get(4))
out = cv2.VideoWriter('genel_4.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 20.0, (w, h))

def check_green_locally(person_img):
    if person_img is None or person_img.size == 0: return False
    ih, iw = person_img.shape[:2]
    upper = person_img[int(ih*0.1):int(ih*0.50), :] # Göğüs bölgesini biraz genişlettim
    hsv = cv2.cvtColor(upper, cv2.COLOR_BGR2HSV)
    # Fosforlu yelek için daha geniş renk aralığı
    mask = cv2.inRange(hsv, np.array([20, 40, 40]), np.array([95, 255, 255]))
    return (np.sum(mask > 0) / mask.size) > GREEN_RATIO_THRESHOLD

frame_idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break

    # A. PANO ANALİZİ
    dst_pts = np.float32([[0, 0], [224, 0], [224, 224], [0, 224]])
    matrix = cv2.getPerspectiveTransform(PANO_PTS, dst_pts)
    pano_crop = cv2.warpPerspective(frame, matrix, (224, 224))
    pano_results = pano_model.predict(pano_crop, verbose=False)[0]
    pano_label = pano_results.names[pano_results.probs.top1]

    # B. PERSONEL TESPİTİ
    results = person_model.predict(frame, conf=PERSON_CONF, imgsz=640, classes=[0], verbose=False)
    
    current_frame_ids = []
    any_authorized_near_pano = False
    any_person_near_pano = False

    # Pano Etki Alanı
    p_min_x, p_max_x = np.min(PANO_PTS[:, 0]) - DISTANCE_OFFSET, np.max(PANO_PTS[:, 0]) + DISTANCE_OFFSET
    p_min_y, p_max_y = np.min(PANO_PTS[:, 1]) - DISTANCE_OFFSET, np.max(PANO_PTS[:, 1]) + DISTANCE_OFFSET

    if len(results[0].boxes) > 0:
        for box in results[0].boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
            
            # ID Takibi (Mesafe toleransı 120px)
            assigned_id = None
            for pid, last_pos in person_history.items():
                if np.sqrt((cx - last_pos[0])**2 + (cy - last_pos[1])**2) < 120:
                    assigned_id = pid
                    break
            if assigned_id is None:
                assigned_id = next_id
                next_id += 1
            
            person_history[assigned_id] = (cx, cy)
            last_boxes[assigned_id] = (x1, y1, x2, y2)
            current_frame_ids.append(assigned_id)

            # YETKİ KONTROLÜ VE SABİTLEME
            # Eğer bu ID daha önce yetki almadıysa kontrol et
            if assigned_id not in authorized_ids:
                crop = frame[max(0, y1):min(h, y2), max(0, x1):min(w, x2)]
                if check_green_locally(crop):
                    authorized_ids.add(assigned_id)
            
            # PANO YAKINLIK KONTROLÜ (Box Kesişimi)
            if not (x2 < p_min_x or x1 > p_max_x or y2 < p_min_y or y1 > p_max_y):
                any_person_near_pano = True
                # ÖNEMLİ: Eğer bu kişi yetkiliyse (ya da hafızadan yetkili geldiyse)
                if assigned_id in authorized_ids:
                    any_authorized_near_pano = True

    # C. KARAR VE ÇİZİM
    
    # 1. Pano Durumu
    pano_color = (0, 255, 0) if pano_label == "kapali" else (0, 0, 255)
    cv2.polylines(frame, [PANO_PTS.astype(np.int32)], True, pano_color, 3)

    # 2. İhlal Kontrolü (Açık + İnsan Var + Yetkili Yok)
    if pano_label == "acik" and any_person_near_pano and not any_authorized_near_pano:
        cv2.putText(frame, "!!! YETKISIZ MUDAHALE !!!", (50, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 4)
        if not pano_ihlal_kaydedildi:
            cv2.imwrite(os.path.join(ihlal_path, f"pano_ihlal_{frame_idx}.jpg"), frame)
            pano_ihlal_kaydedildi = True
    elif pano_label == "kapali" or any_authorized_near_pano:
        pano_ihlal_kaydedildi = False # Yetkili gelirse veya pano kapanırsa ihlal söner

    # 3. Görsel Alanlar
    overlay = frame.copy()
    cv2.fillPoly(overlay, [YASAK_YOL_PTS], (0, 0, 255)) 
    cv2.rectangle(frame, (int(p_min_x), int(p_min_y)), (int(p_max_x), int(p_max_y)), (255, 255, 0), 1) 
    cv2.addWeighted(overlay, 0.15, frame, 0.85, 0, frame)

    # 4. Kutular
    for pid in current_frame_ids:
        x1, y1, x2, y2 = last_boxes[pid]
        is_auth = pid in authorized_ids
        color = (0, 255, 0) if is_auth else (0, 0, 255)
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)
        cv2.putText(frame, f"ID:{pid} {'YETKILI' if is_auth else 'PERSONEL'}", 
                    (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    out.write(frame)
    frame_idx += 1

cap.release()
out.release()
print("✅ Senkronizasyon ve yetki kilidi uygulandı.")

✅ Senkronizasyon ve yetki kilidi uygulandı.


In [2]:
import cv2
import numpy as np
import os
from ultralytics import YOLO

# --- 1. AYARLAR ---
PANO_MODEL_PATH = r"C:\Users\User\runs\classify\pano_denetim_v12\weights\best.pt"
PERSON_MODEL_PATH = 'yolov8n.pt'
VIDEO_PATH = "test_yetkili.mp4"

# Koordinatlar
PANO_PTS = np.float32([[200, 620], [420, 660], [430, 780], [220, 750]])
YASAK_YOL_PTS = np.array([[1400, 10], [1250, 10], [1080, 1080], [1730, 1080]], np.int32)

# Stabilizasyon Ayarları
DISTANCE_OFFSET = 550       # Pano etki alanı genişliği
GREEN_RATIO_THRESHOLD = 0.09 # Yelek hassasiyeti
IHLAL_SAYAC_ESIGI = 5       # Bir durumun ihlal sayılması için kaç kare üst üste olması gerektiği

# Kayıt Klasörü
desktop = os.path.join(os.path.expanduser("~"), "Desktop")
ihlal_path = os.path.join(desktop, "ihlal")
if not os.path.exists(ihlal_path): os.makedirs(ihlal_path)

# Modeller
pano_model = YOLO(PANO_MODEL_PATH)
person_model = YOLO(PERSON_MODEL_PATH)

# Hafıza Üniteleri
authorized_ids = set()      # Bir kez yetki alan bir daha asla kırmızı olmaz
person_history = {}         # ID takibi için
last_boxes = {}             # Çizim stabilitesi için
yasak_yol_kayitli_idlar = set()
pano_ihlal_sayaci = 0       # Yanıp sönmeyi engellemek için
next_id = 1

cap = cv2.VideoCapture(VIDEO_PATH)
w, h = int(cap.get(3)), int(cap.get(4))
out = cv2.VideoWriter('genel_5.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 20.0, (w, h))

def check_green_locally(person_img):
    if person_img is None or person_img.size == 0: return False
    ih, iw = person_img.shape[:2]
    upper = person_img[int(ih*0.1):int(ih*0.55), :] # Üst gövdeye odaklan
    hsv = cv2.cvtColor(upper, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, np.array([25, 40, 40]), np.array([95, 255, 255]))
    return (np.sum(mask > 0) / mask.size) > GREEN_RATIO_THRESHOLD

frame_idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break

    # 1. PANO ANALİZİ
    dst_pts = np.float32([[0, 0], [224, 0], [224, 224], [0, 224]])
    matrix = cv2.getPerspectiveTransform(PANO_PTS, dst_pts)
    pano_crop = cv2.warpPerspective(frame, matrix, (224, 224))
    pano_res = pano_model.predict(pano_crop, verbose=False)[0]
    pano_label = pano_res.names[pano_res.probs.top1]

    # 2. PERSONEL ANALİZİ
    results = person_model.predict(frame, conf=0.40, imgsz=640, classes=[0], verbose=False)
    
    current_frame_ids = []
    any_person_near_pano = False
    any_authorized_near_pano = False

    p_min_x, p_max_x = np.min(PANO_PTS[:, 0]) - DISTANCE_OFFSET, np.max(PANO_PTS[:, 0]) + DISTANCE_OFFSET
    p_min_y, p_max_y = np.min(PANO_PTS[:, 1]) - DISTANCE_OFFSET, np.max(PANO_PTS[:, 1]) + DISTANCE_OFFSET

    if len(results[0].boxes) > 0:
        for box in results[0].boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
            
            # ID Takibi
            assigned_id = None
            for pid, last_pos in person_history.items():
                if np.sqrt((cx - last_pos[0])**2 + (cy - last_pos[1])**2) < 120:
                    assigned_id = pid
                    break
            if assigned_id is None:
                assigned_id = next_id
                next_id += 1
            
            person_history[assigned_id] = (cx, cy)
            last_boxes[assigned_id] = (x1, y1, x2, y2)
            current_frame_ids.append(assigned_id)

            # Yetki Sabitleme (Bir kez yeşilse hep yeşil)
            if assigned_id not in authorized_ids:
                crop = frame[max(0, y1):min(h, y2), max(0, x1):min(w, x2)]
                if check_green_locally(crop):
                    authorized_ids.add(assigned_id)

            # --- İHLAL 1: YASAK YOL ---
            if cv2.pointPolygonTest(YASAK_YOL_PTS, (float(cx), float(y2)), False) >= 0:
                if assigned_id not in yasak_yol_kayitli_idlar:
                    cv2.imwrite(os.path.join(ihlal_path, f"Yol_Ihlal_ID_{assigned_id}.jpg"), frame)
                    yasak_yol_kayitli_idlar.add(assigned_id)
                cv2.putText(frame, "YOL IHLALI!", (x1, y2+20), 1, 1.5, (0,0,255), 2)

            # Pano Yakınlık Kontrolü
            if not (x2 < p_min_x or x1 > p_max_x or y2 < p_min_y or y1 > p_max_y):
                any_person_near_pano = True
                if assigned_id in authorized_ids:
                    any_authorized_near_pano = True

    # --- İHLAL 2: YETKİSİZ PANO MÜDAHALESİ (GECİKTİRMELİ FİLTRE) ---
    if pano_label == "acik" and any_person_near_pano and not any_authorized_near_pano:
        pano_ihlal_sayaci += 1
    else:
        pano_ihlal_sayaci = 0 # Bir anlık bile yetkili gelse veya pano kapansa sayaç sıfırlanır

    if pano_ihlal_sayaci >= IHLAL_SAYAC_ESIGI:
        cv2.putText(frame, "!!! YETKISIZ PANO MUDAHALESI !!!", (50, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 4)
        # Saniyede sadece 1 kez fotoğraf çekmek için frame_idx kontrolü
        if frame_idx % 20 == 0:
            cv2.imwrite(os.path.join(ihlal_path, f"Pano_Ihlal_F_{frame_idx}.jpg"), frame)

    # 3. GÖRSELLEŞTİRME
    pano_color = (0, 255, 0) if pano_label == "kapali" else (0, 0, 255)
    cv2.polylines(frame, [PANO_PTS.astype(np.int32)], True, pano_color, 3)
    
    overlay = frame.copy()
    cv2.fillPoly(overlay, [YASAK_YOL_PTS], (0, 0, 255)) # Yasak Yol
    cv2.rectangle(frame, (int(p_min_x), int(p_min_y)), (int(p_max_x), int(p_max_y)), (255, 255, 0), 1) # Pano Güvenli Bölge
    cv2.addWeighted(overlay, 0.15, frame, 0.85, 0, frame)

    for pid in current_frame_ids:
        x1, y1, x2, y2 = last_boxes[pid]
        is_auth = pid in authorized_ids
        color = (0, 255, 0) if is_auth else (0, 0, 255)
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)
        cv2.putText(frame, f"ID:{pid} {'YETKILI' if is_auth else 'PERSONEL'}", 
                    (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    out.write(frame)
    frame_idx += 1

cap.release()
out.release()
print(f"✅ Sistem optimize edildi. Yol ihlalleri ve Pano ihlalleri Masaüstü/ihlal klasöründe.")

✅ Sistem optimize edildi. Yol ihlalleri ve Pano ihlalleri Masaüstü/ihlal klasöründe.


In [ ]:
import cv2
import numpy as np
import os
import requests
from ultralytics import YOLO

# --- 1. AYARLAR ---
BOT_TOKEN = "" 
CHAT_ID = ""

PANO_MODEL_PATH = r"C:\Users\User\runs\classify\pano_denetim_v12\weights\best.pt"
PERSON_MODEL_PATH = 'yolov8n.pt'
VIDEO_PATH = "test_yetkili.mp4"

# Koordinatlar
PANO_PTS = np.float32([[200, 620], [420, 660], [430, 780], [220, 750]])
YASAK_YOL_PTS = np.array([[1400, 10], [1250, 10], [1080, 1080], [1730, 1080]], np.int32)

# Stabilizasyon Ayarları
DISTANCE_OFFSET = 550
GREEN_RATIO_THRESHOLD = 0.09
IHLAL_SAYAC_ESIGI = 5

# Kayıt Klasörü
desktop = os.path.join(os.path.expanduser("~"), "Desktop")
ihlal_path = os.path.join(desktop, "ihlal")
if not os.path.exists(ihlal_path): os.makedirs(ihlal_path)

# Modeller
pano_model = YOLO(PANO_MODEL_PATH)
person_model = YOLO(PERSON_MODEL_PATH)

# Hafıza Üniteleri
authorized_ids = set()
person_history = {}
last_boxes = {}
yasak_yol_kayitli_idlar = set()
pano_ihlal_sayaci = 0
next_id = 1

def send_telegram_alert(image_path, message):
    """Telegram üzerinden fotoğraf ve mesaj gönderir."""
    try:
        # Mesaj Gönderimi
        msg_url = f"https://api.telegram.org/bot{BOT_TOKEN}/sendMessage"
        requests.post(msg_url, data={"chat_id": CHAT_ID, "text": message}, timeout=5)
        
        # Fotoğraf Gönderimi
        img_url = f"https://api.telegram.org/bot{BOT_TOKEN}/sendPhoto"
        with open(image_path, "rb") as photo:
            requests.post(img_url, data={"chat_id": CHAT_ID}, files={"photo": photo}, timeout=5)
    except Exception as e:
        print(f"Telegram Hatası: {e}")

def check_green_locally(person_img):
    if person_img is None or person_img.size == 0: return False
    ih, iw = person_img.shape[:2]
    upper = person_img[int(ih*0.1):int(ih*0.55), :]
    hsv = cv2.cvtColor(upper, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, np.array([25, 40, 40]), np.array([95, 255, 255]))
    return (np.sum(mask > 0) / mask.size) > GREEN_RATIO_THRESHOLD

cap = cv2.VideoCapture(VIDEO_PATH)
w, h = int(cap.get(3)), int(cap.get(4))
out = cv2.VideoWriter('genel_6_telegram.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 20.0, (w, h))

# Başlangıç Mesajı
requests.post(f"https://api.telegram.org/bot{BOT_TOKEN}/sendMessage", 
              data={"chat_id": CHAT_ID, "text": "🚀 Fabrika İSG Denetim Sistemi Başlatıldı!"})

frame_idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break

    # 1. PANO ANALİZİ
    dst_pts = np.float32([[0, 0], [224, 0], [224, 224], [0, 224]])
    matrix = cv2.getPerspectiveTransform(PANO_PTS, dst_pts)
    pano_crop = cv2.warpPerspective(frame, matrix, (224, 224))
    pano_res = pano_model.predict(pano_crop, verbose=False)[0]
    pano_label = pano_res.names[pano_res.probs.top1]

    # 2. PERSONEL ANALİZİ
    results = person_model.predict(frame, conf=0.40, imgsz=640, classes=[0], verbose=False)
    current_frame_ids = []
    any_person_near_pano = False
    any_authorized_near_pano = False

    p_min_x, p_max_x = np.min(PANO_PTS[:, 0]) - DISTANCE_OFFSET, np.max(PANO_PTS[:, 0]) + DISTANCE_OFFSET
    p_min_y, p_max_y = np.min(PANO_PTS[:, 1]) - DISTANCE_OFFSET, np.max(PANO_PTS[:, 1]) + DISTANCE_OFFSET

    if len(results[0].boxes) > 0:
        for box in results[0].boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
            
            # ID Takibi
            assigned_id = None
            for pid, last_pos in person_history.items():
                if np.sqrt((cx - last_pos[0])**2 + (cy - last_pos[1])**2) < 120:
                    assigned_id = pid
                    break
            if assigned_id is None:
                assigned_id = next_id
                next_id += 1
            
            person_history[assigned_id] = (cx, cy)
            last_boxes[assigned_id] = (x1, y1, x2, y2)
            current_frame_ids.append(assigned_id)

            if assigned_id not in authorized_ids:
                crop = frame[max(0, y1):min(h, y2), max(0, x1):min(w, x2)]
                if check_green_locally(crop):
                    authorized_ids.add(assigned_id)

            # --- İHLAL 1: YASAK YOL ---
            if cv2.pointPolygonTest(YASAK_YOL_PTS, (float(cx), float(y2)), False) >= 0:
                if assigned_id not in yasak_yol_kayitli_idlar:
                    file_p = os.path.join(ihlal_path, f"Yol_Ihlal_ID_{assigned_id}.jpg")
                    cv2.imwrite(file_p, frame)
                    yasak_yol_kayitli_idlar.add(assigned_id)
                    # Telegram Bildirimi
                    send_telegram_alert(file_p, f"⚠️ YOL İHLALİ!\nID: {assigned_id}\nKonum: Yasaklı Yaya Yolu")

            if not (x2 < p_min_x or x1 > p_max_x or y2 < p_min_y or y1 > p_max_y):
                any_person_near_pano = True
                if assigned_id in authorized_ids:
                    any_authorized_near_pano = True

    # --- İHLAL 2: YETKİSİZ PANO MÜDAHALESİ ---
    if pano_label == "acik" and any_person_near_pano and not any_authorized_near_pano:
        pano_ihlal_sayaci += 1
    else:
        pano_ihlal_sayaci = 0

    if pano_ihlal_sayaci == IHLAL_SAYAC_ESIGI: # Tam eşik anında 1 kez gönder
        cv2.putText(frame, "!!! YETKISIZ PANO MUDAHALESI !!!", (50, 100), 1, 2, (0, 0, 255), 3)
        file_p = os.path.join(ihlal_path, f"Pano_Ihlal_F_{frame_idx}.jpg")
        cv2.imwrite(file_p, frame)
        send_telegram_alert(file_p, f"🚨 KRİTİK İHLAL!\nYetkisiz kişi açık panoya müdahale ediyor!")

    # 3. GÖRSELLEŞTİRME
    pano_color = (0, 255, 0) if pano_label == "kapali" else (0, 0, 255)
    cv2.polylines(frame, [PANO_PTS.astype(np.int32)], True, pano_color, 3)
    
    overlay = frame.copy()
    cv2.fillPoly(overlay, [YASAK_YOL_PTS], (0, 0, 255))
    cv2.rectangle(frame, (int(p_min_x), int(p_min_y)), (int(p_max_x), int(p_max_y)), (255, 255, 0), 1)
    cv2.addWeighted(overlay, 0.15, frame, 0.85, 0, frame)

    for pid in current_frame_ids:
        x1, y1, x2, y2 = last_boxes[pid]
        is_auth = pid in authorized_ids
        color = (0, 255, 0) if is_auth else (0, 0, 255)
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)
        cv2.putText(frame, f"ID:{pid} {'YETKILI' if is_auth else 'PERSONEL'}", (x1, y1-10), 1, 1, color, 2)

    out.write(frame)
    frame_idx += 1

cap.release()
out.release()
print("✅ İşlem tamamlandı. Bildirimler Telegram'a iletildi.")

✅ İşlem tamamlandı. Bildirimler Telegram'a iletildi.
